In [1]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

In [2]:
engine = create_engine("sqlite:///mta.db")
inspect(engine).get_table_names()

['mta_data']

In [3]:
pd.read_sql('SELECT * FROM mta_data LIMIT 3;', engine)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,03:00:00,REGULAR,7537255,2570434
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,07:00:00,REGULAR,7537260,2570445
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,11:00:00,REGULAR,7537285,2570500


In [4]:
pd.read_sql('SELECT COUNT(*) FROM mta_data;', engine)

,COUNT(*)
0,2927288


In [5]:
#right time period?
pd.read_sql('SELECT MAX(date), MIN(date) FROM mta_data;', engine)

,MAX(date),MIN(date)
0,06/04/2021,02/27/2021


In [6]:
pd.read_sql('SELECT * FROM mta_data LIMIT 5;', engine)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,03:00:00,REGULAR,7537255,2570434
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,07:00:00,REGULAR,7537260,2570445
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,11:00:00,REGULAR,7537285,2570500
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,15:00:00,REGULAR,7537354,2570546
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,19:00:00,REGULAR,7537501,2570582


In [7]:
#check the CA amounts of each station to understand CA
pd.read_sql('''SELECT station, COUNT(DISTINCT ca) 
               FROM mta_data 
               GROUP BY station 
               ORDER BY COUNT(DISTINCT ca) DESC;''', engine)

,STATION,COUNT(DISTINCT ca)
0,34 ST-PENN STA,13
1,23 ST,13
2,FULTON ST,12
3,86 ST,9
4,WALL ST,8
...,...,...
374,14TH STREET,1
375,138/GRAND CONC,1
376,121 ST,1
377,116 ST-COLUMBIA,1


In [8]:
pd.read_sql('''SELECT SUM(cas) 
               FROM (SELECT station, COUNT(DISTINCT ca) CAs 
                     FROM mta_data 
                     GROUP BY station 
                     ORDER BY COUNT(DISTINCT ca) DESC);''', engine)

,SUM(cas)
0,750


In [9]:
#ca is unigue
pd.read_sql('SELECT COUNT(DISTINCT ca) FROM mta_data;', engine)

,COUNT(DISTINCT ca)
0,750


In [55]:
#explore the unit
#find duplicate units
pd.read_sql('''SELECT unit, COUNT(DISTINCT station) 
               FROM mta_data 
               GROUP BY unit 
               ORDER BY COUNT(DISTINCT station) desc;''', engine)

,UNIT,COUNT(DISTINCT station)
0,R549,4
1,R029,3
2,R550,2
3,R541,2
4,R540,2
...,...,...
464,R007,1
465,R006,1
466,R005,1
467,R004,1


In [56]:
pd.read_sql('''SELECT unit, COUNT(DISTINCT ca) 
               FROM mta_data 
               GROUP BY unit 
               ORDER BY COUNT(DISTINCT ca) desc;''', engine)

,UNIT,COUNT(DISTINCT ca)
0,R305,6
1,R028,6
2,R195,5
3,R170,5
4,R163,5
...,...,...
464,R008,1
465,R007,1
466,R005,1
467,R004,1


In [47]:
pd.read_sql('''SELECT SUM(units) 
               FROM (SELECT station, COUNT(DISTINCT unit) UNITs 
                     FROM mta_data 
                     GROUP BY station 
                     ORDER BY COUNT(DISTINCT unit) DESC);''', engine)

,SUM(units)
0,485


In [70]:
#station X ca
pd.read_sql('''SELECT ca, COUNT(DISTINCT station) 
               FROM mta_data 
               GROUP BY ca 
               ORDER BY COUNT(DISTINCT station) DESC;''', engine)

,CA,COUNT(DISTINCT station)
0,A002,1
1,A006,1
2,A007,1
3,A010,1
4,A011,1
...,...,...
745,S101,1
746,S101A,1
747,S102,1
748,TRAM1,1


In [53]:
#CA X UNIT and find one CA only with one UNIT
pd.read_sql('''SELECT ca, station, COUNT(DISTINCT unit) 
               FROM mta_data 
               GROUP BY ca 
               ORDER BY COUNT(DISTINCT unit) DESC;''', engine)

,CA,STATION,COUNT(DISTINCT unit)
0,A002,59 ST,1
1,A006,5 AV/59 ST,1
2,A007,5 AV/59 ST,1
3,A010,57 ST-7 AV,1
4,A011,57 ST-7 AV,1
...,...,...,...
745,S101,ST. GEORGE,1
746,S101A,ST. GEORGE,1
747,S102,TOMPKINSVILLE,1
748,TRAM1,RIT-MANHATTAN,1


In [68]:
#CA X SCP
pd.read_sql('''SELECT ca, station, COUNT(DISTINCT scp) 
               FROM mta_data 
               GROUP BY ca 
               ORDER BY COUNT(DISTINCT scp) DESC;''', engine)

,CA,STATION,COUNT(DISTINCT scp)
0,PTH22,PATH NEW WTC,45
1,PTH02,HARRISON,28
2,PTH03,JOURNAL SQUARE,26
3,R610,ATL AV-BARCLAY,25
4,PTH07,CITY / BUS,23
...,...,...,...
745,R727,PELHAM PKWY,3
746,R729,BAYCHESTER AV,3
747,H037,SUTTER AV,2
748,J022,CLEVELAND ST,2


In [10]:
#check the SCP amounts of each station to understand SCP
pd.read_sql('''SELECT station, COUNT(DISTINCT scp) 
               FROM mta_data 
               GROUP BY station 
               ORDER BY COUNT(DISTINCT scp) DESC;''', engine)

,STATION,COUNT(DISTINCT scp)
0,FULTON ST,80
1,34 ST-PENN STA,63
2,GRD CNTRL-42 ST,53
3,ATL AV-BARCLAY,51
4,CHAMBERS ST,50
...,...,...
374,190 ST,3
375,182-183 STS,3
376,138/GRAND CONC,3
377,SUTTER AV,2


In [11]:
#found scp number may be duplicate among stations
pd.read_sql('SELECT COUNT(DISTINCT scp) FROM mta_data;', engine)

,COUNT(DISTINCT scp)
0,224


In [25]:
#compare SCP at "ZEREGA AV" between 3/1/2021 and 5/31/2021
pd.read_sql('''SELECT * FROM mta_data 
               WHERE station = "ZEREGA AV"
               AND date = "03/01/2021"
               ORDER BY station, time, scp
               LIMIT 6;''', engine)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,R419,R326,00-00-00,ZEREGA AV,6,IRT,03/01/2021,00:00:00,REGULAR,167346,684997
1,R419,R326,00-00-01,ZEREGA AV,6,IRT,03/01/2021,00:00:00,REGULAR,226262,370060
2,R419,R326,00-03-00,ZEREGA AV,6,IRT,03/01/2021,00:00:00,REGULAR,1140077,517750
3,R419,R326,00-03-01,ZEREGA AV,6,IRT,03/01/2021,00:00:00,REGULAR,1213195,231884
4,R419,R326,00-05-00,ZEREGA AV,6,IRT,03/01/2021,00:00:00,REGULAR,232,0
5,R419,R326,00-05-01,ZEREGA AV,6,IRT,03/01/2021,00:00:00,REGULAR,39,171


In [27]:
#find two rare-used SCPs: 00-05-00 and 00-05-01
pd.read_sql('''SELECT * FROM mta_data 
               WHERE station = "ZEREGA AV"
               AND date = "05/31/2021"
               ORDER BY station, time desc, scp
               LIMIT 6;''', engine)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,R419,R326,00-00-00,ZEREGA AV,6,IRT,05/31/2021,21:00:00,REGULAR,180441,743927
1,R419,R326,00-00-01,ZEREGA AV,6,IRT,05/31/2021,21:00:00,REGULAR,236204,388908
2,R419,R326,00-03-00,ZEREGA AV,6,IRT,05/31/2021,21:00:00,REGULAR,1157583,529251
3,R419,R326,00-03-01,ZEREGA AV,6,IRT,05/31/2021,21:00:00,REGULAR,1254260,242087
4,R419,R326,00-05-00,ZEREGA AV,6,IRT,05/31/2021,21:00:00,REGULAR,236,0
5,R419,R326,00-05-01,ZEREGA AV,6,IRT,05/31/2021,21:00:00,REGULAR,39,179


In [40]:
#explore the linename
#linename needs to be seperate #use pandas to do it later if needed
pd.read_sql('SELECT linename, COUNT(DISTINCT station) FROM mta_data GROUP BY linename ORDER BY linename;', engine)

,LINENAME,COUNT(DISTINCT station)
0,1,48
1,123,4
2,1237ACENQRS,1
3,1237ACENQRSW,1
4,123ACE,1
...,...,...
109,R1W,1
110,R2345,1
111,RNW,1
112,S,1


In [57]:
pd.read_sql('''SELECT station, COUNT(DISTINCT linename) 
               FROM mta_data 
               GROUP BY station 
               ORDER BY COUNT(DISTINCT linename) DESC;''', engine)

,STATION,COUNT(DISTINCT linename)
0,86 ST,5
1,23 ST,5
2,125 ST,4
3,TIMES SQ-42 ST,3
4,KINGS HWY,3
...,...,...
374,121 ST,1
375,116 ST-COLUMBIA,1
376,110 ST,1
377,103 ST-CORONA,1


In [62]:
#check multiple linenames in a station
pd.read_sql('SELECT station, linename, COUNT(DISTINCT ca) FROM mta_data WHERE station = "86 ST" GROUP BY station, linename ;', engine)

,STATION,LINENAME,COUNT(DISTINCT ca)
0,86 ST,1,3
1,86 ST,456,2
2,86 ST,BC,1
3,86 ST,N,1
4,86 ST,R,2


In [63]:
pd.read_sql('SELECT COUNT(DISTINCT ca) FROM mta_data WHERE station = "86 ST";', engine)

,COUNT(DISTINCT ca)
0,9


In [65]:
#how about the busy station: 34 ST-PENN STA
pd.read_sql('''SELECT station, linename, ca, COUNT(DISTINCT scp) 
               FROM mta_data 
               WHERE station = "34 ST-PENN STA" 
               GROUP BY station, linename, ca ;''', engine)

,STATION,LINENAME,CA,COUNT(DISTINCT scp)
0,34 ST-PENN STA,123,R135,6
1,34 ST-PENN STA,123,R137,3
2,34 ST-PENN STA,123,R139,5
3,34 ST-PENN STA,123,R141,7
4,34 ST-PENN STA,123ACE,R138,19
5,34 ST-PENN STA,123ACE,R142,6
6,34 ST-PENN STA,ACE,N067,15
7,34 ST-PENN STA,ACE,N068,4
8,34 ST-PENN STA,ACE,N069,4
9,34 ST-PENN STA,ACE,N070,7


In [67]:
#how about the busy station: FULTON ST
pd.read_sql('''SELECT station, linename, ca, COUNT(DISTINCT scp) 
               FROM mta_data 
               WHERE station = "FULTON ST" 
               GROUP BY station, linename, ca ;''', engine)

,STATION,LINENAME,CA,COUNT(DISTINCT scp)
0,FULTON ST,2345ACJZ,N095A,13
1,FULTON ST,2345ACJZ,N098,12
2,FULTON ST,2345ACJZ,R113,8
3,FULTON ST,2345ACJZ,R114,4
4,FULTON ST,2345ACJZ,R205A,8
5,FULTON ST,2345ACJZ,R206,9
6,FULTON ST,2345ACJZ,R208,9
7,FULTON ST,ACJZ2345,A077,4
8,FULTON ST,ACJZ2345,A081,7
9,FULTON ST,ACJZ2345,A082,4


In [41]:
#explore the division
#should only have BMT, IRT, or IND?
pd.read_sql('SELECT division, COUNT(DISTINCT station) FROM mta_data GROUP BY division ORDER BY division;', engine)

,DIVISION,COUNT(DISTINCT station)
0,BMT,120
1,IND,149
2,IRT,157
3,PTH,19
4,RIT,2
5,SRT,2


In [12]:
#would like to understand the impact of DESC
pd.read_sql('SELECT desc FROM mta_data GROUP BY desc;', engine)

,DESC
0,RECOVR AUD
1,REGULAR


In [13]:
#checking some detailed data of "RECOVR AUD"
pd.read_sql('''SELECT * 
               FROM mta_data 
               WHERE desc = "RECOVR AUD"
               ORDER BY station, date, time;''', engine)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,H007A,R248,02-03-04,1 AV,L,BMT,02/27/2021,07:00:00,RECOVR AUD,354769,57885
1,H007A,R248,02-00-03,1 AV,L,BMT,03/17/2021,08:00:00,RECOVR AUD,81991,170354
2,H007A,R248,02-01-01,1 AV,L,BMT,03/17/2021,08:00:00,RECOVR AUD,1,37
3,H007A,R248,02-03-00,1 AV,L,BMT,03/17/2021,08:00:00,RECOVR AUD,63269,805034
4,H007A,R248,02-03-01,1 AV,L,BMT,03/17/2021,08:00:00,RECOVR AUD,84040,457154
...,...,...,...,...,...,...,...,...,...,...,...
10643,R419,R326,00-00-01,ZEREGA AV,6,IRT,03/03/2021,08:00:00,RECOVR AUD,226501,370583
10644,R419,R326,00-03-00,ZEREGA AV,6,IRT,03/03/2021,08:00:00,RECOVR AUD,1140679,518035
10645,R419,R326,00-03-01,ZEREGA AV,6,IRT,03/03/2021,08:00:00,RECOVR AUD,1214389,232138
10646,R419,R326,00-05-00,ZEREGA AV,6,IRT,03/03/2021,08:00:00,RECOVR AUD,232,0


In [20]:
#use "ZEREGA AV" as an example to exam the accruacy of "RECOVR AUD" data
#find out the data of desc is good to include
pd.read_sql('''SELECT * FROM mta_data 
               WHERE station = "ZEREGA AV"
               AND date = "03/03/2021"
               ORDER BY station, date, time;''', engine)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,R419,R326,00-00-00,ZEREGA AV,6,IRT,03/03/2021,00:00:00,REGULAR,167639,686423
1,R419,R326,00-00-01,ZEREGA AV,6,IRT,03/03/2021,00:00:00,REGULAR,226475,370560
2,R419,R326,00-03-00,ZEREGA AV,6,IRT,03/03/2021,00:00:00,REGULAR,1140591,518016
3,R419,R326,00-03-01,ZEREGA AV,6,IRT,03/03/2021,00:00:00,REGULAR,1214213,232120
4,R419,R326,00-05-00,ZEREGA AV,6,IRT,03/03/2021,00:00:00,REGULAR,232,0
5,R419,R326,00-05-01,ZEREGA AV,6,IRT,03/03/2021,00:00:00,REGULAR,39,171
6,R419,R326,00-00-00,ZEREGA AV,6,IRT,03/03/2021,04:00:00,REGULAR,167641,686434
7,R419,R326,00-00-01,ZEREGA AV,6,IRT,03/03/2021,04:00:00,REGULAR,226475,370564
8,R419,R326,00-03-00,ZEREGA AV,6,IRT,03/03/2021,04:00:00,REGULAR,1140593,518023
9,R419,R326,00-03-01,ZEREGA AV,6,IRT,03/03/2021,04:00:00,REGULAR,1214215,232123


## Conclusion from SQL analysis
### Needed data
* The code of a control area is unique. A control area will only have one unit and multiple SCPs but units may be duplicate among stations. Linename is messy. => Just use Control Area
* Time is not consistent too. => Use morning/afternoon/night...
* Can keep DESC
### To do
* Substract entries and exits to get the numbers
* Date and time analysis